In [5]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits import mplot3d
import matplotlib.gridspec as gridspec
from neo.core import SpikeTrain
import elephant as eph
import quantities as q
import numpy as np
import os
import re 
import seaborn as sns
import pandas as pd
import imageio
from subprocess import call
mpl.rcParams.update({'font.size': 15})
%config Completer.use_jedi = False

In [2]:
def eval_spikes(graphno,binsize=200):
    mat = np.loadtxt(f'../modules/networks/matrix_{graphno}.csv',delimiter=",")
    module = np.loadtxt(f'../modules/networks/matrix_{graphno}_modules.csv')
    order = np.argsort(module)
    all_traces = []
    for pertseed in [59428,13674,84932,72957,85036]:
        traces = np.load(f"../fig2/Data/LN30_data_{graphno}_{pertseed}.npy",allow_pickle=True)
        traces = np.append(traces,np.median(traces)*np.ones((5,1,30)),axis=1)
        traces = np.apply_along_axis(lambda v: np.convolve(v,1000*np.ones(binsize)/binsize,)[:v.shape[0]],1,np.logical_and(traces[:,:-1,:]<-20,traces[:,1:,:]>=-20))
        all_traces.append(traces)
    all_traces = np.concatenate(all_traces,axis=1)
    axs = []
    for i in np.unique(module):
        temp = np.mean(all_traces[:,:,np.arange(30)[module==i]],axis=2)
        axs.append(temp)
    axs = np.transpose(np.array(axs),(1,0,2))
    return axs
def order_by_variability(axs):
    axs = axs[:,np.argsort(axs.std(axis=(0,2)))[::-1],:]
    return axs, [f"M{mod+1}" for mod in np.argsort(axs.std(axis=(0,2)))]

In [12]:
def animate3d(data,graphno,nreps=5,labels=["C1","C2","C3"]):
    for rep in range(nreps):
        print(f"Replicate {rep+1}")
        lims = axs[rep][0].min(),axs[rep][0].max(),axs[rep][1].min(),axs[rep][1].max(),axs[rep][2].min(),axs[rep][2].max()
        fig = plt.figure(figsize=(5,7))
        G = gridspec.GridSpec(5, 1)

        ax1 = fig.add_subplot(G[0:4], projection='3d')
        ax1.text(-0.1, -0.1, -0.1, "0", color='k',fontdict={"fontsize":10})
        ax1.quiver(0,0,0,lims[1],0,0,arrow_length_ratio=0.1,color='k')
        ax1.text(lims[1]+0.05, -0.05, -0.05, labels[0] , color='k',fontdict={"fontsize":10})
        ax1.quiver(0,0,0,0,lims[3],0,arrow_length_ratio=0.1,color='k')
        ax1.text(-0.05,lims[3]+0.05,-0.05, labels[1], color='k',fontdict={"fontsize":10})
        ax1.quiver(0,0,0,0,0,lims[5],arrow_length_ratio=0.1,color='k')
        ax1.text(-0.05,-0.05,lims[5]+0.05, labels[2], color='k',fontdict={"fontsize":10})
        ax1.set_xlim(lims[0],lims[1])
        ax1.set_ylim(lims[2],lims[3])
        ax1.set_zlim(lims[4],lims[5])
        ax1.set_axis_off()
        ax1.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax1.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax1.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
        ax1.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax1.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
        ax1.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)

        line = ax1.plot3D(0, 0, 0, 'gray',linewidth=1)
        dot = ax1.plot3D(0, 0, 0,marker='o',color='k')

        tim = ax1.text2D(0.4,0.05,"t=0.0s",transform=ax1.transAxes,fontsize=10)
        
        plt.title(f"Replicate {rep+1}")
        
        ax2 = fig.add_subplot(G[4])
        tracks = []
        endpoint = []
        for i in range(axs.shape[1]):
            a = ax2.plot(np.arange(0,0.05*1,0.05),axs[rep][0][0],"-",
                         color=plt.cm.inferno(i/axs.shape[1]),linewidth=0.5,label=f'C{i+1}')
            tracks.append(a)
            a = ax2.plot(0,0,"o",color=plt.cm.inferno(i/axs.shape[1]))
            endpoint.append(a)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
                  ncol=axs.shape[1], fontsize=10, frameon=False)
        ax2.set_xlim((0,35))
        ax2.set_ylim((0,1.2*np.max([lims[1],lims[3],lims[5]])))
        ax2.set_xlabel("Time (in s)")
        ax2.set_ylabel("Mean FR (Hz)")
        
        if rep == 2:
            plt.suptitle(f"Random Network {graphno}")
        
        perts = []
        for i in range(5):
            for j in range(5):
                perts.append(7*i+1.5+j)
        pert_counter = 0

        plt.subplots_adjust(wspace=0.0,hspace=0.0)

        for i in tqdm(range(500,axs.shape[2],50)):
            tim.set_text(f"t = {i/1000:0.3f}s")
            tim.set_color("black")
            if pert_counter<len(perts) and perts[pert_counter]<i/1000:
                ax2.fill_betweenx((0,1.2*np.max([lims[1],lims[3],lims[5]])),
                                  perts[pert_counter],perts[pert_counter]+0.1,color='k',alpha=0.1)
                pert_counter+=1
                tim.set_text(f"t = {i/1000:0.3f} s")
                tim.set_color("red")
            xline = axs[rep][0][i-500:i]
            yline = axs[rep][1][i-500:i]
            zline = axs[rep][2][i-500:i]
            line[0].set_data_3d(xline, yline, zline)
            dot[0].set_data_3d(xline[-1], yline[-1], zline[-1])
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)
            ax1.view_init(elev=20, azim=i*0.02)
            for j in range(axs.shape[1]):
                tracks[j][0].set_xdata(np.arange(i)/1000)
                tracks[j][0].set_ydata(axs[rep][j][:i])
                endpoint[j][0].set_xdata(i/1000)
                endpoint[j][0].set_ydata(axs[rep][j][i])
            plt.savefig(f"anim/{i}.png", dpi=90)
        plt.close()
        
        images = []
        files = os.listdir('anim')
        files.sort(key=lambda f: int(re.sub('\D', '', f)))
        for filename in tqdm(files):
            images.append(imageio.imread('anim/'+filename))
        imageio.mimsave(f'Videos/movie_{graphno}_{rep}.gif', images,fps=20)
        
        for file in os.listdir('anim/'):
            os.remove('anim/'+file)

def animate2d(data,graphno,nreps=5,labels=["M1","M2"],rep=1):
    for rep in range(nreps):
        print(f"Replicate {rep+1}")
        lims = axs[rep][0].min(),axs[rep][0].max(),axs[rep][1].min(),axs[rep][1].max()
        fig = plt.figure(figsize=(5,7))
        G = gridspec.GridSpec(5, 1)

        ax1 = fig.add_subplot(G[0:4])
        ax1.set_xlim(lims[0],lims[1])
        ax1.set_ylim(lims[2],lims[3])
        ax1.set_xticks([lims[0],lims[1]])
        ax1.set_xticklabels(["0",labels[0]])
        ax1.set_yticks([lims[2],lims[3]])
        ax1.set_yticklabels(["",labels[1]])
        line = ax1.plot(0, 0, 'gray',linewidth=1)
        dot = ax1.plot(0, 0, marker='o',color='k')

        tim = ax1.text(0.4,0.05,"t=0.0s",transform=ax1.transAxes,fontsize=10)
        ax1.spines['right'].set_visible(False)
        ax1.spines['top'].set_visible(False)
        plt.title(f"Replicate {rep+1}")
        
        ax2 = fig.add_subplot(G[4])
        tracks = []
        endpoint = []
        for i in range(axs.shape[1]):
            a = ax2.plot(np.arange(0,0.05*1,0.05),axs[rep][0][0],"-",
                         color=plt.cm.inferno(i/axs.shape[1]),linewidth=0.5,label=f'C{i+1}')
            tracks.append(a)
            a = ax2.plot(0,0,"o",color=plt.cm.inferno(i/axs.shape[1]))
            endpoint.append(a)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.05),
                  ncol=3, fontsize=10, frameon=False)
        ax2.set_xlim((0,35))
        ax2.set_ylim((0,1.2*np.max([lims[1],lims[3]])))
        ax2.set_xlabel("Time (in s)")
        ax2.set_ylabel("Mean FR (Hz)")
        
        if rep == 2:
            plt.suptitle(f"Random Network {graphno}")
        
        perts = []
        for i in range(5):
            for j in range(5):
                perts.append(7*i+1.5+j)
        pert_counter = 0

        plt.subplots_adjust(wspace=0.0,hspace=0.5)

        for i in tqdm(range(500,axs.shape[2],50)):
            tim.set_text(f"t = {i/1000:0.3f}s")
            tim.set_color("black")
            if pert_counter<len(perts) and perts[pert_counter]<i/1000:
                ax2.fill_betweenx((0,1.2*np.max([lims[1],lims[3]])),perts[pert_counter],perts[pert_counter]+0.1,color='k',alpha=0.1)
                pert_counter+=1
                tim.set_text(f"t = {i/1000:0.3f} s")
                tim.set_color("red")
            xline = axs[rep][0][i-500:i]
            yline = axs[rep][1][i-500:i]
            line[0].set_data(xline, yline)
            dot[0].set_data(xline[-1], yline[-1])
            plt.gca().spines['top'].set_visible(False)
            plt.gca().spines['right'].set_visible(False)
            for j in range(axs.shape[1]):
                tracks[j][0].set_xdata(np.arange(i)/1000)
                tracks[j][0].set_ydata(axs[rep][j][:i])
                endpoint[j][0].set_xdata(i/1000)
                endpoint[j][0].set_ydata(axs[rep][j][i])
            plt.savefig(f"anim/{i}.png", dpi=90)
        plt.close()
        
        images = []
        files = os.listdir('anim')
        files.sort(key=lambda f: int(re.sub('\D', '', f)))
        for filename in tqdm(files):
            images.append(imageio.imread('anim/'+filename))
        imageio.mimsave(f'Videos/movie_{graphno}_{rep}.gif', images,fps=20)
        
        for file in os.listdir('anim/'):
            os.remove('anim/'+file)

In [13]:
axs = eval_spikes(1)
print(axs.shape)
axs,labels = order_by_variability(axs)
animate3d(axs,1,labels=labels)

  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

(5, 4, 34860)
Replicate 1


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

Replicate 2


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

...

  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

Replicate 4


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:03<00:00, 178.48it/s]


In [14]:
axs = eval_spikes(2)
print(axs.shape)
animate2d(axs,2)

  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

(5, 2, 34860)
Replicate 1


  0%|                                                                                  | 1/688 [00:00<01:31,  7.48it/s]

Replicate 2


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:31,  7.53it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:22,  8.28it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:03<00:00, 189.38it/s]


In [15]:
axs = eval_spikes(3)
print(axs.shape)
animate2d(axs,3)

  0%|                                                                                  | 1/688 [00:00<01:30,  7.60it/s]

(5, 2, 34860)
Replicate 1


  0%|                                                                                  | 1/688 [00:00<01:39,  6.90it/s]

Replicate 2


  0%|                                                                                  | 1/688 [00:00<01:24,  8.09it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:28,  7.79it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:34,  7.24it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:04<00:00, 155.59it/s]


In [16]:
axs = eval_spikes(4)
print(axs.shape)
animate3d(axs,4)

  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

(5, 3, 34860)
Replicate 1


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

Replicate 2


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

...

  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:44,  6.56it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:04<00:00, 169.80it/s]


In [17]:
axs = eval_spikes(5)
print(axs.shape)
animate3d(axs,5)

  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

(5, 3, 34860)
Replicate 1


  0%|                                                                                  | 1/688 [00:00<01:39,  6.92it/s]

Replicate 2


  0%|                                                                                  | 1/688 [00:00<01:46,  6.47it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:40,  6.82it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:44,  6.57it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:03<00:00, 185.41it/s]


In [18]:
axs = eval_spikes(6)
print(axs.shape)
animate2d(axs,6)

  0%|                                                                                  | 1/688 [00:00<01:38,  7.00it/s]

(5, 2, 34860)
Replicate 1


  0%|                                                                                  | 1/688 [00:00<01:30,  7.59it/s]

Replicate 2


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:30,  7.61it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:32,  7.42it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:03<00:00, 172.84it/s]


In [19]:
axs = eval_spikes(7)
print(axs.shape)
animate3d(axs,7)

  0%|                                                                                  | 1/688 [00:00<01:48,  6.31it/s]

(5, 3, 34860)
Replicate 1


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

Replicate 2


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:47,  6.39it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:43,  6.63it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:04<00:00, 168.00it/s]


In [20]:
axs = eval_spikes(8)
print(axs.shape)
animate2d(axs,8)

  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

(5, 2, 34860)
Replicate 1


  0%|                                                                                  | 1/688 [00:00<01:32,  7.43it/s]

Replicate 2


  0%|                                                                                  | 1/688 [00:00<01:37,  7.01it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:31,  7.54it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:50,  6.23it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:03<00:00, 189.30it/s]


In [21]:
axs = eval_spikes(9)
print(axs.shape)
animate3d(axs,9)

  0%|                                                                                  | 1/688 [00:00<01:44,  6.55it/s]

(5, 3, 34860)
Replicate 1


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

Replicate 2


  0%|                                                                                          | 0/688 [00:00<?, ?it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:40,  6.86it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:42,  6.68it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:03<00:00, 181.38it/s]


In [22]:
axs = eval_spikes(10)
print(axs.shape)
animate2d(axs,10)

  0%|                                                                                  | 1/688 [00:00<01:28,  7.73it/s]

(5, 2, 34860)
Replicate 1


  0%|                                                                                  | 1/688 [00:00<01:30,  7.60it/s]

Replicate 2


  0%|                                                                                  | 1/688 [00:00<01:34,  7.27it/s]

...

  0%|                                                                                  | 1/688 [00:00<01:29,  7.64it/s]

Replicate 4


  0%|                                                                                  | 1/688 [00:00<01:34,  7.30it/s]

Replicate 5


100%|███████████████████████████████████████████████████████████████████████████████| 688/688 [00:03<00:00, 190.52it/s]


In [23]:
for graphno in range(1,11):
    gifs = [imageio.get_reader(f'Videos/{x}') for x in filter(lambda v: f"_{graphno}_" in v,os.listdir("Videos/"))]
    new_gif = imageio.get_writer(f'Videos/{graphno}_out.gif',fps=20)
    for frame_number in tqdm(range(np.min([gif.get_length() for gif in gifs]))):
        datas = [gif.get_next_data() for gif in gifs]
        new_image = np.hstack(datas)
        new_gif.append_data(new_image)
    for gif in gifs:
        gif.close()
    new_gif.close()

100%|████████████████████████████████████████████████████████████████████████████████| 688/688 [01:29<00:00,  7.70it/s]


In [9]:
files = list(filter(lambda v: "_out" in v,os.listdir("Videos/")))
files.sort(key=lambda f: int(re.sub('\D', '', f)))
for graphno in range(1,11,2):
    gifs = [imageio.get_reader(f'Videos/{x}') for x in files[graphno-1:graphno+1]]
    new_gif = imageio.get_writer(f'Videos/{graphno}_{graphno+1}.gif',fps=20)
    for frame_number in tqdm(range(np.min([gif.get_length() for gif in gifs]))):
        datas = [gif.get_next_data() for gif in gifs]
        new_image = np.vstack(datas)
        new_gif.append_data(new_image)
    for gif in gifs:
        gif.close()
    new_gif.close()

100%|████████████████████████████████████████████████████████████████████████████████| 688/688 [03:27<00:00,  3.32it/s]


ValueError: zero-size array to reduction operation minimum which has no identity

In [7]:
for graphno in tqdm(range(1,11,2)):
    call(['D:\\Program Files (x86)\\FFMPEG\\bin\\ffmpeg.exe','-i',
          f'Videos/{graphno}_{graphno+1}.gif',f'Videos/{graphno}_{graphno+1}.mp4'])